In [ ]:
import json
import torch
import torch.nn.functional as F
import os
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AutoModelForCausalLM

# =============================================================================
# [1] 설정
# =============================================================================
WORK_DIR = "/content/drive/MyDrive/P02_SemanticParsing"
BASE_DIR = f"{WORK_DIR}/nia"
TABLE_PATH = f"{BASE_DIR}/tables.json"
VALID_PATH = f"{BASE_DIR}/valid.json"
SAVE_PATH = f"{WORK_DIR}/final_results_batched.json"

LINKER_PATH = f"{WORK_DIR}/saved_models_final/schema_linker_e5/epoch_1"
RERANKER_PATH = f"{WORK_DIR}/saved_models_final/reranker_v5_final/epoch_1"
GENERATOR_NAME = "defog/sqlcoder-7b-2"

BATCH_SIZE = 32  # ★ A100 40GB면 32~64 추천 (OOM 나면 줄이세요)
device = "cuda"

print(f"🔥 [Speed-Up] 배치 파이프라인 가동 (Batch Size: {BATCH_SIZE})")

# =============================================================================
# [2] 모델 로드
# =============================================================================
print("📂 모델 로드 중...")
linker_tok = AutoTokenizer.from_pretrained(LINKER_PATH)
linker_mod = AutoModel.from_pretrained(LINKER_PATH).to(device).eval()

# Reranker
try:
    rerank_tok = AutoTokenizer.from_pretrained(RERANKER_PATH)
    rerank_mod = AutoModelForSequenceClassification.from_pretrained(RERANKER_PATH).to(device).eval()
    use_reranker = True
except:
    use_reranker = False
    print("⚠️ Reranker Skip")

# Generator (SQLCoder)
gen_tok = AutoTokenizer.from_pretrained(GENERATOR_NAME, padding_side='left') # ★ Left Padding 필수
gen_tok.pad_token = gen_tok.eos_token # Pad 토큰 설정
gen_mod = AutoModelForCausalLM.from_pretrained(
    GENERATOR_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

# =============================================================================
# [3] 유틸리티 (인덱싱 & 프롬프트) - 기존과 동일
# =============================================================================
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def build_index(table_list):
    col_texts, col_metas = [], []
    print("⚡ 컬럼 인덱싱 생성 중...")
    for t in table_list:
        t_en = t.get('table_names_original', [""])[0]
        t_ko = t.get('table_names', [""])[0]
        if isinstance(t_ko, list): t_ko = t_ko[0]

        for i, (c_en, c_ko) in enumerate(zip(t['column_names_original'], t['column_names'])):
            if c_en[1] == "*": continue
            p_text = f"passage: {t_en}.{c_en[1]}({c_ko[1]})"
            r_text = f"테이블: {t_ko} ({t_en}) | 컬럼: {c_ko[1]} ({c_en[1]})"

            col_texts.append(p_text)
            col_metas.append({
                "raw": p_text, "rerank": r_text, "t_en": t_en,
                "c_en": c_en[1], "c_ko": c_ko[1], "t_ko": t_ko, "type": t['column_types'][i]
            })
    return col_texts, col_metas

def make_prompt_strict(question, top_cols):
    if not top_cols: return f"SELECT * FROM error;" # 예외 처리
    table_scores = {}
    for c in top_cols:
        t_en = c['t_en']
        table_scores[t_en] = table_scores.get(t_en, 0) + 1
    best_table = max(table_scores, key=table_scores.get)
    filtered_cols = [c for c in top_cols if c['t_en'] == best_table]

    cols_ddl = [f"   {c['c_en']} {c['type']}, -- {c['c_ko']}" for c in filtered_cols]
    cols_str = ",\n".join(cols_ddl)
    t_desc = filtered_cols[0]['t_ko']
    schema_str = f"CREATE TABLE {best_table} ( -- {t_desc}\n{cols_str}\n);"

    prompt = f"""### Instructions
Your task is to generate a valid SQLite SQL query to answer the question.
1. **Use ONLY the provided table in the Schema.** Do NOT hallucinate or join with tables not listed below.
2. If the column is TEXT type, use `LIKE` for pattern matching instead of date functions.
3. Select ONLY the columns requested in the question.

### Database Schema
{schema_str}

### Question
{question}

### Answer
```sql
"""
    return prompt

# =============================================================================
# [4] 인덱싱 (미리 수행)
# =============================================================================
with open(TABLE_PATH, 'r') as f: tables = json.load(f)
col_texts, col_metas = build_index(tables)

col_embs = []
with torch.no_grad():
    for i in tqdm(range(0, len(col_texts), 256), desc="Embedding Index"): # 인덱싱은 배치 크게
        batch = col_texts[i:i+256]
        inp = linker_tok(batch, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
        out = linker_mod(**inp)
        emb = mean_pooling(out, inp['attention_mask'])
        col_embs.append(F.normalize(emb, p=2, dim=1).cpu())
col_embs = torch.cat(col_embs, dim=0)

# =============================================================================
# [5] 메인 루프 (배치 처리 적용)
# =============================================================================
with open(VALID_PATH, 'r') as f: valid_data = json.load(f)
# valid_data = valid_data[:100] # 테스트용

results = []
print(f"\n🚀 총 {len(valid_data)}개 배치 처리 시작...")

for i in tqdm(range(0, len(valid_data), BATCH_SIZE), desc="Processing Batches"):
    batch_items = valid_data[i:i+BATCH_SIZE]
    batch_questions = [item['question'] for item in batch_items]

    # 1. E5 Retrieval (Batch)
    # -----------------------
    q_texts = [f"query: {q}" for q in batch_questions]
    q_inp = linker_tok(q_texts, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)

    with torch.no_grad():
        q_emb = F.normalize(mean_pooling(linker_mod(**q_inp), q_inp['attention_mask']), p=2, dim=1).cpu()

    # Matrix Multiplication (Batch x All_Cols)
    scores = torch.matmul(q_emb, col_embs.T)
    top_k_vals, top_k_inds = torch.topk(scores, k=20, dim=1) # (B, 20)

    batch_prompts = []
    batch_final_cands = []

    # 2. Reranking & Prompt Build (Per Item in Batch)
    # -----------------------------------------------
    # Reranker는 로직이 복잡해서 내부 루프로 돌려도 A100에선 순식간임 (병목 아님)
    for b_idx in range(len(batch_items)):
        question = batch_questions[b_idx]
        cand_inds = top_k_inds[b_idx].tolist()
        candidates = [col_metas[idx] for idx in cand_inds]

        if use_reranker:
            # Reranker 배치 처리 (Top-20 * 1 = 20개씩)
            r_inputs = [cand['rerank'] for cand in candidates]
            # (질문, 후보) 쌍 만들기
            pairs = [[question, cand_text] for cand_text in r_inputs]

            # 여기서 작은 배치 추론
            r_toks = rerank_tok(pairs, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
            with torch.no_grad():
                logits = rerank_mod(**r_toks).logits[:, 1] # Positive score

            # 점수 결합
            cands_with_score = sorted(zip(candidates, logits.tolist()), key=lambda x: x[1], reverse=True)
            final_cands = [x[0] for x in cands_with_score[:5]]
        else:
            final_cands = candidates[:5]

        batch_final_cands.append(final_cands)
        # 프롬프트 생성
        batch_prompts.append(make_prompt_strict(question, final_cands))

    # 3. SQLCoder Generation (Batch) ★ 여기가 속도의 핵심
    # ----------------------------------------------------
    gen_inputs = gen_tok(batch_prompts, return_tensors="pt", padding=True, truncation=True).to(device)

    with torch.no_grad():
        gen_outputs = gen_mod.generate(
            **gen_inputs,
            max_new_tokens=200,
            do_sample=False,
            num_beams=1,
            eos_token_id=gen_tok.eos_token_id,
            pad_token_id=gen_tok.eos_token_id
        )

    # 4. Decoding & Saving
    # --------------------
    decoded_preds = gen_tok.batch_decode(gen_outputs, skip_special_tokens=True)

    for b_idx, pred_text in enumerate(decoded_preds):
        # 프롬프트 제거하고 SQL만 추출
        try:
            generated_sql = pred_text.split("```sql")[-1].split("```")[0].strip()
        except:
            generated_sql = pred_text # 파싱 실패 시 원문

        if "SELECT" in generated_sql:
            generated_sql = "SELECT" + generated_sql.split("SELECT", 1)[1]

        item = batch_items[b_idx]
        final_cands = batch_final_cands[b_idx]

        results.append({
            "db_id": item['db_id'],
            "question": item['question'],
            "gold_sql": item['query'],
            "pred_sql": generated_sql,
            "chosen_table": final_cands[0]['t_en'] if final_cands else "None"
        })

# 저장
with open(SAVE_PATH, 'w') as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

print(f"✅ 완료! 저장됨: {SAVE_PATH}")


시연용

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 파일명: 30_demo_showcase_final_v5.ipynb
import json
import torch
import torch.nn.functional as F
import time
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AutoModelForCausalLM

# =============================================================================
# [1] 설정 & 경로
# =============================================================================
WORK_DIR = "/content/drive/MyDrive/P02_SemanticParsing"
BASE_DIR = f"{WORK_DIR}/nia"
TABLE_PATH = f"{BASE_DIR}/tables.json"
VALID_PATH = f"{BASE_DIR}/valid.json"

# 모델 경로
LINKER_PATH = f"{WORK_DIR}/saved_models_final/schema_linker_e5/epoch_1"
RERANKER_PATH = f"{WORK_DIR}/saved_models_final/reranker_v5_final/epoch_1"
GENERATOR_NAME = "defog/sqlcoder-7b-2"

device = "cuda" if torch.cuda.is_available() else "cpu"

print("🎬 [Demo] Text-to-SQL 파이프라인 (Final V5 - Dictator Mode)")
print(f"   - Device: {device}")
print(f"   - Model: {GENERATOR_NAME}")
print("-" * 60)

# =============================================================================
# [2] 모델 로드
# =============================================================================
print("📂 모델 로드 중...")
linker_tok = AutoTokenizer.from_pretrained(LINKER_PATH)
linker_mod = AutoModel.from_pretrained(LINKER_PATH).to(device).eval()

try:
    rerank_tok = AutoTokenizer.from_pretrained(RERANKER_PATH)
    rerank_mod = AutoModelForSequenceClassification.from_pretrained(RERANKER_PATH).to(device).eval()
    use_reranker = True
except:
    use_reranker = False
    print("   ⚠️ Reranker 로드 실패 (E5 단독 실행)")

gen_tok = AutoTokenizer.from_pretrained(GENERATOR_NAME)
gen_mod = AutoModelForCausalLM.from_pretrained(
    GENERATOR_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)
print("✅ 준비 완료! (Prompt V5 Applied)")
print("=" * 60 + "\n")

# =============================================================================
# [3] 핵심 유틸리티 & 독재자 프롬프트 (V5)
# =============================================================================
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def make_prompt_demo_v5(question, top_cols):
    if not top_cols: return ""

    # 1등 테이블 선정
    table_scores = {}
    for c in top_cols:
        t_en = c['t_en']
        table_scores[t_en] = table_scores.get(t_en, 0) + 1
    best_table = max(table_scores, key=table_scores.get)

    filtered_cols = [c for c in top_cols if c['t_en'] == best_table]

    # DDL 생성 (영어 힌트 포함 - 모델이 헷갈리지 않게)
    cols_ddl = []
    for c in filtered_cols:
        korean_desc = c['c_ko']
        hint = ""
        # 모델이 한글 컬럼명을 영어로 맘대로 바꾸지 못하게 힌트 제공
        if "명" in korean_desc or "이름" in korean_desc: hint = " (Use this for Name)"
        elif "주소" in korean_desc: hint = " (Use this for Address)"
        elif "일시" in korean_desc: hint = " (Use this for Time)"

        cols_ddl.append(f"   {c['c_en']} {c['type']}, -- {korean_desc}{hint}")

    cols_str = ",\n".join(cols_ddl)
    t_desc = filtered_cols[0]['t_ko']
    schema_str = f"CREATE TABLE {best_table} ( -- {t_desc}\n{cols_str}\n);"

    # ★ V5 프롬프트: "명령 불복종 금지" ★
    prompt = f"""### Instructions
Your task is to generate a valid SQLite SQL query.

**MANDATORY RULES (DO NOT IGNORE):**
1. **Single Table Only:** Use `{best_table}`. Do NOT join with external tables like TB_PHARMACY.
2. **Column Name Enforcement:**
   - For Pharmacy Names, you MUST use `DUTYNAME` (or the provided name column). NEVER use `NAME`.
   - For Addresses, you MUST use `DUTYADDR` (or the provided address column). NEVER use `ADDRESS`.
   - For Times, you MUST use `WORK_DTTM` (or the provided time column). NEVER use `TIME`.
3. **Logic:**
   - To find an address by name: `SELECT DUTYADDR ... WHERE DUTYNAME LIKE '%Name%'`
4. **Day of Week Mapping:**
   - Monday->`DUTYTIME1...`, Tuesday->`DUTYTIME2...`, etc.
   - Do NOT use `EXTRACT` or date functions. Just select the columns.

### Database Schema
{schema_str}

### Question
{question}

### Answer
```sql
"""
    return prompt

# =============================================================================
# [4] 인덱싱 (미리 수행)
# =============================================================================
with open(TABLE_PATH, 'r') as f: tables = json.load(f)
col_texts, col_metas = [], []
for t in tables:
    t_en = t.get('table_names_original', [""])[0]
    t_ko = t.get('table_names', [""])[0]
    if isinstance(t_ko, list): t_ko = t_ko[0]
    for i, (c_en, c_ko) in enumerate(zip(t['column_names_original'], t['column_names'])):
        if c_en[1] == "*": continue
        p_text = f"passage: {t_en}.{c_en[1]}({c_ko[1]})"
        r_text = f"테이블: {t_ko} ({t_en}) | 컬럼: {c_ko[1]} ({c_en[1]})"
        col_texts.append(p_text)
        col_metas.append({"raw": p_text, "rerank": r_text, "t_en": t_en, "c_en": c_en[1], "c_ko": c_ko[1], "t_ko": t_ko, "type": t['column_types'][i]})

# 배치 인덱싱
col_embs = []
with torch.no_grad():
    for i in range(0, len(col_texts), 256):
        batch = col_texts[i:i+256]
        inp = linker_tok(batch, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
        out = linker_mod(**inp)
        emb = mean_pooling(out, inp['attention_mask'])
        col_embs.append(F.normalize(emb, p=2, dim=1).cpu())
col_embs = torch.cat(col_embs, dim=0)

# =============================================================================
# [5] 시연 루프 (Top 10 Showcase)
# =============================================================================
with open(VALID_PATH, 'r') as f: valid_data = json.load(f)[:10]

print("🚀 [Live Demo] Generating SQL...")
print("=" * 70)

for i, item in enumerate(valid_data):
    question = item['question']
    start_time = time.time()

    # 1. Search
    q_inp = linker_tok(f"query: {question}", return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    with torch.no_grad():
        q_emb = F.normalize(mean_pooling(linker_mod(**q_inp), q_inp['attention_mask']), p=2, dim=1).cpu()
    scores = torch.matmul(q_emb, col_embs.T)[0]
    top20_idx = torch.topk(scores, k=20).indices.tolist()
    candidates = [col_metas[idx] for idx in top20_idx]

    # 2. Rerank
    if use_reranker:
        r_scores = []
        with torch.no_grad():
            for cand in candidates:
                inp = rerank_tok(question, cand['rerank'], return_tensors="pt", truncation=True, max_length=128).to(device)
                r_scores.append(rerank_mod(**inp).logits[0][1].item())
        final_cands = [x[0] for x in sorted(zip(candidates, r_scores), key=lambda x: x[1], reverse=True)[:5]]
    else:
        final_cands = candidates[:5]

    # 3. Generate (With V5 Prompt)
    full_prompt = make_prompt_demo_v5(question, final_cands)
    gen_inp = gen_tok(full_prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        gen_out = gen_mod.generate(**gen_inp, max_new_tokens=150, do_sample=False, num_beams=1)

    sql = gen_tok.decode(gen_out[0], skip_special_tokens=True).split("```sql")[-1].split("```")[0].strip()
    if "SELECT" in sql: sql = "SELECT" + sql.split("SELECT", 1)[1]

    duration = time.time() - start_time

    # 시연용 출력 (SQL 컬러 강조)
    print(f"\n[Question {i+1}] {question}")
    print(f" ➤ Detected Table: {final_cands[0]['t_en']} ({final_cands[0]['t_ko']})")
    # Cyan Color for SQL
    print(f" ➤ Generated SQL: \033[1;96m{sql}\033[0m")
    print(f" ➤ Processing Time: {duration:.2f} sec")
    print("-" * 70)

    # 영상에서 읽을 시간 확보
    time.sleep(0.5)

print("\n✨ Demo Finished! (Perfect Version)")


In [ ]:
# 파일명: 29_demo_showcase_final_v4.ipynb
import json
import torch
import torch.nn.functional as F
import time
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AutoModelForCausalLM

# =============================================================================
# [1] 설정 & 경로
# =============================================================================
WORK_DIR = "/content/drive/MyDrive/P02_SemanticParsing"
BASE_DIR = f"{WORK_DIR}/nia"
TABLE_PATH = f"{BASE_DIR}/tables.json"
VALID_PATH = f"{BASE_DIR}/valid.json"

LINKER_PATH = f"{WORK_DIR}/saved_models_final/schema_linker_e5/epoch_1"
RERANKER_PATH = f"{WORK_DIR}/saved_models_final/reranker_v5_final/epoch_1"
GENERATOR_NAME = "defog/sqlcoder-7b-2"

device = "cuda" if torch.cuda.is_available() else "cpu"

print("🎬 [Demo] Text-to-SQL 파이프라인 (Final V4 - Perfect)")
print(f"   - Device: {device}")
print(f"   - Model: {GENERATOR_NAME}")
print("-" * 60)

# =============================================================================
# [2] 모델 로드
# =============================================================================
print("📂 모델 로드 중...")
linker_tok = AutoTokenizer.from_pretrained(LINKER_PATH)
linker_mod = AutoModel.from_pretrained(LINKER_PATH).to(device).eval()

try:
    rerank_tok = AutoTokenizer.from_pretrained(RERANKER_PATH)
    rerank_mod = AutoModelForSequenceClassification.from_pretrained(RERANKER_PATH).to(device).eval()
    use_reranker = True
except:
    use_reranker = False
    print("   ⚠️ Reranker 로드 실패 (E5 단독 실행)")

gen_tok = AutoTokenizer.from_pretrained(GENERATOR_NAME)
gen_mod = AutoModelForCausalLM.from_pretrained(
    GENERATOR_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)
print("✅ 준비 완료! (Prompt V4 Applied)")
print("=" * 60 + "\n")

# =============================================================================
# [3] 유틸리티 & 프롬프트 (V4: 요일 매핑 치트키 추가)
# =============================================================================
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def make_prompt_demo_v4(question, top_cols):
    if not top_cols: return ""

    # 1등 테이블 선정
    table_scores = {}
    for c in top_cols:
        t_en = c['t_en']
        table_scores[t_en] = table_scores.get(t_en, 0) + 1
    best_table = max(table_scores, key=table_scores.get)

    filtered_cols = [c for c in top_cols if c['t_en'] == best_table]
    cols_ddl = [f"   {c['c_en']} {c['type']}, -- {c['c_ko']}" for c in filtered_cols]
    cols_str = ",\n".join(cols_ddl)
    t_desc = filtered_cols[0]['t_ko']

    schema_str = f"CREATE TABLE {best_table} ( -- {t_desc}\n{cols_str}\n);"

    # ★ V4 프롬프트: 요일 매핑 규칙 명시 ★
    prompt = f"""### Instructions
Your task is to generate a valid SQLite SQL query to answer the question.

**CRITICAL RULES:**
1. **NO JOINs:** Use ONLY `{best_table}`. Do NOT join with other tables.
2. **Day of Week Mapping (Important):**
   - Monday (월) -> `DUTYTIME1...`
   - Tuesday (화) -> `DUTYTIME2...`
   - Wednesday (수) -> `DUTYTIME3...`
   - Thursday (목) -> `DUTYTIME4...`
   - Friday (금) -> `DUTYTIME5...`
   - Saturday (토) -> `DUTYTIME6...`
   - Sunday (일) -> `DUTYTIME7...`
   - **Do NOT calculate dates.** Just select the corresponding columns.
3. **Pattern Matching:** If the column is TEXT type, use `LIKE`.

### Database Schema
{schema_str}

### Question
{question}

### Answer
```sql
"""
    return prompt

# =============================================================================
# [4] 인덱싱 (미리 수행)
# =============================================================================
with open(TABLE_PATH, 'r') as f: tables = json.load(f)
col_texts, col_metas = [], []
for t in tables:
    t_en = t.get('table_names_original', [""])[0]
    t_ko = t.get('table_names', [""])[0]
    if isinstance(t_ko, list): t_ko = t_ko[0]
    for i, (c_en, c_ko) in enumerate(zip(t['column_names_original'], t['column_names'])):
        if c_en[1] == "*": continue
        p_text = f"passage: {t_en}.{c_en[1]}({c_ko[1]})"
        r_text = f"테이블: {t_ko} ({t_en}) | 컬럼: {c_ko[1]} ({c_en[1]})"
        col_texts.append(p_text)
        col_metas.append({"raw": p_text, "rerank": r_text, "t_en": t_en, "c_en": c_en[1], "c_ko": c_ko[1], "t_ko": t_ko, "type": t['column_types'][i]})

# 배치 인덱싱
col_embs = []
with torch.no_grad():
    for i in range(0, len(col_texts), 256):
        batch = col_texts[i:i+256]
        inp = linker_tok(batch, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
        out = linker_mod(**inp)
        emb = mean_pooling(out, inp['attention_mask'])
        col_embs.append(F.normalize(emb, p=2, dim=1).cpu())
col_embs = torch.cat(col_embs, dim=0)

# =============================================================================
# [5] 시연 루프 (Top 10)
# =============================================================================
with open(VALID_PATH, 'r') as f: valid_data = json.load(f)[:10]

print("🚀 [Live Demo] Generating SQL...")
print("=" * 70)

for i, item in enumerate(valid_data):
    question = item['question']
    start_time = time.time()

    # 1. Search
    q_inp = linker_tok(f"query: {question}", return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    with torch.no_grad():
        q_emb = F.normalize(mean_pooling(linker_mod(**q_inp), q_inp['attention_mask']), p=2, dim=1).cpu()
    scores = torch.matmul(q_emb, col_embs.T)[0]
    top20_idx = torch.topk(scores, k=20).indices.tolist()
    candidates = [col_metas[idx] for idx in top20_idx]

    # 2. Rerank
    if use_reranker:
        r_scores = []
        with torch.no_grad():
            for cand in candidates:
                inp = rerank_tok(question, cand['rerank'], return_tensors="pt", truncation=True, max_length=128).to(device)
                r_scores.append(rerank_mod(**inp).logits[0][1].item())
        final_cands = [x[0] for x in sorted(zip(candidates, r_scores), key=lambda x: x[1], reverse=True)[:5]]
    else:
        final_cands = candidates[:5]

    # 3. Generate (With V4 Prompt)
    full_prompt = make_prompt_demo_v4(question, final_cands)
    gen_inp = gen_tok(full_prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        gen_out = gen_mod.generate(**gen_inp, max_new_tokens=150, do_sample=False, num_beams=1)

    sql = gen_tok.decode(gen_out[0], skip_special_tokens=True).split("```sql")[-1].split("```")[0].strip()
    if "SELECT" in sql: sql = "SELECT" + sql.split("SELECT", 1)[1]

    duration = time.time() - start_time

    # 예쁘게 출력
    print(f"\n[Question {i+1}] {question}")
    print(f" ➤ Detected Table: {final_cands[0]['t_en']} ({final_cands[0]['t_ko']})")
    # 파란색 볼드체로 SQL 강조
    print(f" ➤ Generated SQL: \033[1;94m{sql}\033[0m")
    print(f" ➤ Processing Time: {duration:.2f} sec")
    print("-" * 70)

    time.sleep(0.5)

print("\n✨ Demo Finished! (Perfect Version)")


🎬 [Demo] Text-to-SQL 파이프라인 (Final V4 - Perfect)
   - Device: cuda
   - Model: defog/sqlcoder-7b-2
------------------------------------------------------------
📂 모델 로드 중...


The tokenizer you are loading from '/content/drive/MyDrive/P02_SemanticParsing/saved_models_final/schema_linker_e5/epoch_1' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ 준비 완료! (Prompt V4 Applied)

🚀 [Live Demo] Generating SQL...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...


[Question 8] 가로수길약국의 월요일과 화요일의 진료 시간을 알려줘
 ➤ Detected Table: TB_PHARMACY_OPERATE_INFO (서울시 약국 운영시간 정보)
 ➤ Generated SQL: SELECT p.DUTYTIME1S, p.DUTYTIME2C FROM TB_PHARMACY_OPERATE_INFO p WHERE p.DUTYTIME1S IS NOT NULL AND p.DUTYTIME2C IS NOT NULL;
 ➤ Processing Time: 2.30 sec
----------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



[Question 9] 주소에 강남대로가 들어가는 약국의 이름을 알려줘
 ➤ Detected Table: TB_PHARMACY_OPERATE_INFO (서울시 약국 운영시간 정보)
 ➤ Generated SQL: SELECT p.DUTYNAME FROM TB_PHARMACY_OPERATE_INFO p WHERE p.DUTYADDR ilike '%강남대로%'
 ➤ Processing Time: 1.61 sec
----------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



[Question 10] 주소에 동광로가 들어가는 구심약국의 작업 일시를 알려줘
 ➤ Detected Table: TB_LIVE_PRIOR_PARKING_ZONE (서울시 거주자우선주차정보 표준 데이터)
 ➤ Generated SQL: SELECT t.WORK_DTTM FROM TB_LIVE_PRIOR_PARKING_ZONE t WHERE t.ADDRESS ilike '%동광로%'
 ➤ Processing Time: 1.63 sec
----------------------------------------------------------------------

✨ Demo Finished! (Perfect Version)


In [ ]:
# 파일명: 30_demo_showcase_perfect.ipynb
import json
import torch
import torch.nn.functional as F
import time
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AutoModelForCausalLM

# =============================================================================
# [1] 설정
# =============================================================================
WORK_DIR = "/content/drive/MyDrive/P02_SemanticParsing"
BASE_DIR = f"{WORK_DIR}/nia"
TABLE_PATH = f"{BASE_DIR}/tables.json"
VALID_PATH = f"{BASE_DIR}/valid.json"

LINKER_PATH = f"{WORK_DIR}/saved_models_final/schema_linker_e5/epoch_1"
RERANKER_PATH = f"{WORK_DIR}/saved_models_final/reranker_v5_final/epoch_1"
GENERATOR_NAME = "defog/sqlcoder-7b-2"

device = "cuda" if torch.cuda.is_available() else "cpu"

print("🎬 [Demo] Text-to-SQL 파이프라인 (Perfect Version)")
print(f"   - Device: {device}")
print(f"   - Generator: {GENERATOR_NAME}")
print("-" * 60)

# =============================================================================
# [2] 모델 로드
# =============================================================================
print("📂 모델 로드 중... (잠시만 기다려주세요)")
linker_tok = AutoTokenizer.from_pretrained(LINKER_PATH)
linker_mod = AutoModel.from_pretrained(LINKER_PATH).to(device).eval()

try:
    rerank_tok = AutoTokenizer.from_pretrained(RERANKER_PATH)
    rerank_mod = AutoModelForSequenceClassification.from_pretrained(RERANKER_PATH).to(device).eval()
    use_reranker = True
except:
    use_reranker = False
    print("   ⚠️ Reranker 로드 실패 (E5 단독 실행)")

gen_tok = AutoTokenizer.from_pretrained(GENERATOR_NAME)
gen_mod = AutoModelForCausalLM.from_pretrained(
    GENERATOR_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)
print("✅ 준비 완료! (English Hint Strategy Applied)")
print("=" * 60 + "\n")

# =============================================================================
# [3] 핵심: 영어 힌트 주입 & 강력한 프롬프트
# =============================================================================
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def make_prompt_demo_perfect(question, top_cols):
    if not top_cols: return ""

    # 1. 1등 테이블 선정
    table_scores = {}
    for c in top_cols:
        t_en = c['t_en']
        table_scores[t_en] = table_scores.get(t_en, 0) + 1
    best_table = max(table_scores, key=table_scores.get)

    filtered_cols = [c for c in top_cols if c['t_en'] == best_table]

    # 2. ★ DDL 생성 시 영어 힌트(English Hint) 주입 ★
    # 한글 컬럼 설명을 영어로 번역해주면 LLM 이해도가 200% 상승함
    cols_ddl = []
    for c in filtered_cols:
        k_desc = c['c_ko']
        hint = ""
        if "명" in k_desc or "이름" in k_desc: hint = " (Name)"
        elif "주소" in k_desc: hint = " (Address)"
        elif "전화" in k_desc: hint = " (Phone)"
        elif "일시" in k_desc or "날짜" in k_desc: hint = " (Date/Time)"
        elif "시간" in k_desc: hint = " (Time)"

        cols_ddl.append(f"   {c['c_en']} {c['type']}, -- {k_desc}{hint}")

    cols_str = ",\n".join(cols_ddl)
    t_desc = filtered_cols[0]['t_ko']

    schema_str = f"CREATE TABLE {best_table} ( -- {t_desc}\n{cols_str}\n);"

    # 3. 프롬프트: 요일 매핑 & 단일 테이블 강제
    prompt = f"""### Instructions
Your task is to generate a valid SQLite SQL query to answer the question.

**CRITICAL RULES:**
1. **Single Table:** All information (Name, Address, Phone, Time) is in `{best_table}`. Do NOT join with external tables.
2. **Column Mapping:**
   - Search for Names in Name columns (marked as Name).
   - Search for Addresses in Address columns (marked as Address).
   - **Weekdays:** Monday(`DUTYTIME1..`), Tuesday(`DUTYTIME2..`), ..., Sunday(`DUTYTIME7..`).
3. **No Date Functions:** Do NOT use `EXTRACT` or `to_char`. Just select the columns.

### Database Schema
{schema_str}

### Question
{question}

### Answer
```sql
"""
    return prompt

# =============================================================================
# [4] 인덱싱 (Hidden Process)
# =============================================================================
with open(TABLE_PATH, 'r') as f: tables = json.load(f)
col_texts, col_metas = [], []
for t in tables:
    t_en = t.get('table_names_original', [""])[0]
    t_ko = t.get('table_names', [""])[0]
    if isinstance(t_ko, list): t_ko = t_ko[0]
    for i, (c_en, c_ko) in enumerate(zip(t['column_names_original'], t['column_names'])):
        if c_en[1] == "*": continue
        p_text = f"passage: {t_en}.{c_en[1]}({c_ko[1]})"
        r_text = f"테이블: {t_ko} ({t_en}) | 컬럼: {c_ko[1]} ({c_en[1]})"
        col_texts.append(p_text)
        col_metas.append({"raw": p_text, "rerank": r_text, "t_en": t_en, "c_en": c_en[1], "c_ko": c_ko[1], "t_ko": t_ko, "type": t['column_types'][i]})

col_embs = []
with torch.no_grad():
    for i in range(0, len(col_texts), 256):
        batch = col_texts[i:i+256]
        inp = linker_tok(batch, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
        out = linker_mod(**inp)
        emb = mean_pooling(out, inp['attention_mask'])
        col_embs.append(F.normalize(emb, p=2, dim=1).cpu())
col_embs = torch.cat(col_embs, dim=0)

# =============================================================================
# [5] 시연 루프 (Showcase)
# =============================================================================
with open(VALID_PATH, 'r') as f: valid_data = json.load(f)[:10]

print("🚀 [Live Demo] Natural Language to SQL Generation")
print("=" * 70)

for i, item in enumerate(valid_data):
    question = item['question']
    start_time = time.time()

    # 1. Search
    q_inp = linker_tok(f"query: {question}", return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    with torch.no_grad():
        q_emb = F.normalize(mean_pooling(linker_mod(**q_inp), q_inp['attention_mask']), p=2, dim=1).cpu()
    scores = torch.matmul(q_emb, col_embs.T)[0]
    top20_idx = torch.topk(scores, k=20).indices.tolist()
    candidates = [col_metas[idx] for idx in top20_idx]

    # 2. Rerank
    if use_reranker:
        r_scores = []
        with torch.no_grad():
            for cand in candidates:
                inp = rerank_tok(question, cand['rerank'], return_tensors="pt", truncation=True, max_length=128).to(device)
                r_scores.append(rerank_mod(**inp).logits[0][1].item())
        final_cands = [x[0] for x in sorted(zip(candidates, r_scores), key=lambda x: x[1], reverse=True)[:5]]
    else:
        final_cands = candidates[:5]

    # 3. Generate
    full_prompt = make_prompt_demo_perfect(question, final_cands)
    gen_inp = gen_tok(full_prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        gen_out = gen_mod.generate(**gen_inp, max_new_tokens=150, do_sample=False, num_beams=1)

    sql = gen_tok.decode(gen_out[0], skip_special_tokens=True).split("```sql")[-1].split("```")[0].strip()
    if "SELECT" in sql: sql = "SELECT" + sql.split("SELECT", 1)[1]

    duration = time.time() - start_time

    # Output Styling
    print(f"\n[Question {i+1}] {question}")
    print(f" ➤ Detected Table: {final_cands[0]['t_en']} ({final_cands[0]['t_ko']})")
    # Cyan Color + Bold
    print(f" ➤ Generated SQL: \033[1;96m{sql}\033[0m")
    print(f" ➤ Processing Time: {duration:.2f} sec")
    print("-" * 70)

    time.sleep(1.0) # 시연용 딜레이

print("\n✨ Demo Finished!")


🎬 [Demo] Text-to-SQL 파이프라인 (Perfect Version)
   - Device: cuda
   - Generator: defog/sqlcoder-7b-2
------------------------------------------------------------
📂 모델 로드 중... (잠시만 기다려주세요)


The tokenizer you are loading from '/content/drive/MyDrive/P02_SemanticParsing/saved_models_final/schema_linker_e5/epoch_1' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ 준비 완료! (English Hint Strategy Applied)

🚀 [Live Demo] Natural Language to SQL Generation


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...


[Question 8] 가로수길약국의 월요일과 화요일의 진료 시간을 알려줘
 ➤ Detected Table: TB_PHARMACY_OPERATE_INFO (서울시 약국 운영시간 정보)
 ➤ Generated SQL: SELECT p.DUTYTIME1S AS MONDAY_START_TIME, p.DUTYTIME1C AS MONDAY_END_TIME, p.DUTYTIME2C AS TUESDAY_END_TIME FROM TB_PHARMACY_OPERATE_INFO p WHERE p.NAME = '가로수길약국';
 ➤ Processing Time: 3.23 sec
----------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



[Question 9] 주소에 강남대로가 들어가는 약국의 이름을 알려줘
 ➤ Detected Table: TB_PHARMACY_OPERATE_INFO (서울시 약국 운영시간 정보)
 ➤ Generated SQL: SELECT p.NAME FROM TB_PHARMACY_OPERATE_INFO p WHERE p.DUTYADDR ilike '%강남대로%'
 ➤ Processing Time: 1.54 sec
----------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



[Question 10] 주소에 동광로가 들어가는 구심약국의 작업 일시를 알려줘
 ➤ Detected Table: TB_LIVE_PRIOR_PARKING_ZONE (서울시 거주자우선주차정보 표준 데이터)
 ➤ Generated SQL: SELECT lppz.WORK_DTTM FROM TB_LIVE_PRIOR_PARKING_ZONE lppz WHERE lppz.ADDRESS ilike '%동광로%' ORDER BY lppz.WORK_DTTM NULLS LAST;
 ➤ Processing Time: 2.38 sec
----------------------------------------------------------------------

✨ Demo Finished!


In [ ]:
# 파일명: 30_demo_showcase_final_v5.ipynb
import json
import torch
import torch.nn.functional as F
import time
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AutoModelForCausalLM

# =============================================================================
# [1] 설정 & 경로
# =============================================================================
WORK_DIR = "/content/drive/MyDrive/P02_SemanticParsing"
BASE_DIR = f"{WORK_DIR}/nia"
TABLE_PATH = f"{BASE_DIR}/tables.json"
VALID_PATH = f"{BASE_DIR}/valid.json"

# 모델 경로
LINKER_PATH = f"{WORK_DIR}/saved_models_final/schema_linker_e5/epoch_1"
RERANKER_PATH = f"{WORK_DIR}/saved_models_final/reranker_v5_final/epoch_1"
GENERATOR_NAME = "defog/sqlcoder-7b-2"

device = "cuda" if torch.cuda.is_available() else "cpu"

print("🎬 [Demo] Text-to-SQL 파이프라인 (Final V5 - Dictator Mode)")
print(f"   - Device: {device}")
print(f"   - Model: {GENERATOR_NAME}")
print("-" * 60)

# =============================================================================
# [2] 모델 로드
# =============================================================================
print("📂 모델 로드 중...")
linker_tok = AutoTokenizer.from_pretrained(LINKER_PATH)
linker_mod = AutoModel.from_pretrained(LINKER_PATH).to(device).eval()

try:
    rerank_tok = AutoTokenizer.from_pretrained(RERANKER_PATH)
    rerank_mod = AutoModelForSequenceClassification.from_pretrained(RERANKER_PATH).to(device).eval()
    use_reranker = True
except:
    use_reranker = False
    print("   ⚠️ Reranker 로드 실패 (E5 단독 실행)")

gen_tok = AutoTokenizer.from_pretrained(GENERATOR_NAME)
gen_mod = AutoModelForCausalLM.from_pretrained(
    GENERATOR_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)
print("✅ 준비 완료! (Prompt V5 Applied)")
print("=" * 60 + "\n")

# =============================================================================
# [3] 핵심 유틸리티 & 독재자 프롬프트 (V5)
# =============================================================================
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def make_prompt_demo_v5(question, top_cols):
    if not top_cols: return ""

    # 1등 테이블 선정
    table_scores = {}
    for c in top_cols:
        t_en = c['t_en']
        table_scores[t_en] = table_scores.get(t_en, 0) + 1
    best_table = max(table_scores, key=table_scores.get)

    filtered_cols = [c for c in top_cols if c['t_en'] == best_table]

    # DDL 생성 (영어 힌트 포함 - 모델이 헷갈리지 않게)
    cols_ddl = []
    for c in filtered_cols:
        korean_desc = c['c_ko']
        hint = ""
        # 모델이 한글 컬럼명을 영어로 맘대로 바꾸지 못하게 힌트 제공
        if "명" in korean_desc or "이름" in korean_desc: hint = " (Use this for Name)"
        elif "주소" in korean_desc: hint = " (Use this for Address)"
        elif "일시" in korean_desc: hint = " (Use this for Time)"

        cols_ddl.append(f"   {c['c_en']} {c['type']}, -- {korean_desc}{hint}")

    cols_str = ",\n".join(cols_ddl)
    t_desc = filtered_cols[0]['t_ko']
    schema_str = f"CREATE TABLE {best_table} ( -- {t_desc}\n{cols_str}\n);"

    # ★ V5 프롬프트: "명령 불복종 금지" ★
    prompt = f"""### Instructions
Your task is to generate a valid SQLite SQL query.

**MANDATORY RULES (DO NOT IGNORE):**
1. **Single Table Only:** Use `{best_table}`. Do NOT join with external tables like TB_PHARMACY.
2. **Column Name Enforcement:**
   - For Pharmacy Names, you MUST use `DUTYNAME` (or the provided name column). NEVER use `NAME`.
   - For Addresses, you MUST use `DUTYADDR` (or the provided address column). NEVER use `ADDRESS`.
   - For Times, you MUST use `WORK_DTTM` (or the provided time column). NEVER use `TIME`.
3. **Logic:**
   - To find an address by name: `SELECT DUTYADDR ... WHERE DUTYNAME LIKE '%Name%'`
4. **Day of Week Mapping:**
   - Monday->`DUTYTIME1...`, Tuesday->`DUTYTIME2...`, etc.
   - Do NOT use `EXTRACT` or date functions. Just select the columns.

### Database Schema
{schema_str}

### Question
{question}

### Answer
```sql
"""
    return prompt

# =============================================================================
# [4] 인덱싱 (미리 수행)
# =============================================================================
with open(TABLE_PATH, 'r') as f: tables = json.load(f)
col_texts, col_metas = [], []
for t in tables:
    t_en = t.get('table_names_original', [""])[0]
    t_ko = t.get('table_names', [""])[0]
    if isinstance(t_ko, list): t_ko = t_ko[0]
    for i, (c_en, c_ko) in enumerate(zip(t['column_names_original'], t['column_names'])):
        if c_en[1] == "*": continue
        p_text = f"passage: {t_en}.{c_en[1]}({c_ko[1]})"
        r_text = f"테이블: {t_ko} ({t_en}) | 컬럼: {c_ko[1]} ({c_en[1]})"
        col_texts.append(p_text)
        col_metas.append({"raw": p_text, "rerank": r_text, "t_en": t_en, "c_en": c_en[1], "c_ko": c_ko[1], "t_ko": t_ko, "type": t['column_types'][i]})

# 배치 인덱싱
col_embs = []
with torch.no_grad():
    for i in range(0, len(col_texts), 256):
        batch = col_texts[i:i+256]
        inp = linker_tok(batch, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
        out = linker_mod(**inp)
        emb = mean_pooling(out, inp['attention_mask'])
        col_embs.append(F.normalize(emb, p=2, dim=1).cpu())
col_embs = torch.cat(col_embs, dim=0)

# =============================================================================
# [5] 시연 루프 (Top 10 Showcase)
# =============================================================================
with open(VALID_PATH, 'r') as f: valid_data = json.load(f)[:10]

print("🚀 [Live Demo] Generating SQL...")
print("=" * 70)

for i, item in enumerate(valid_data):
    question = item['question']
    start_time = time.time()

    # 1. Search
    q_inp = linker_tok(f"query: {question}", return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    with torch.no_grad():
        q_emb = F.normalize(mean_pooling(linker_mod(**q_inp), q_inp['attention_mask']), p=2, dim=1).cpu()
    scores = torch.matmul(q_emb, col_embs.T)[0]
    top20_idx = torch.topk(scores, k=20).indices.tolist()
    candidates = [col_metas[idx] for idx in top20_idx]

    # 2. Rerank
    if use_reranker:
        r_scores = []
        with torch.no_grad():
            for cand in candidates:
                inp = rerank_tok(question, cand['rerank'], return_tensors="pt", truncation=True, max_length=128).to(device)
                r_scores.append(rerank_mod(**inp).logits[0][1].item())
        final_cands = [x[0] for x in sorted(zip(candidates, r_scores), key=lambda x: x[1], reverse=True)[:5]]
    else:
        final_cands = candidates[:5]

    # 3. Generate (With V5 Prompt)
    full_prompt = make_prompt_demo_v5(question, final_cands)
    gen_inp = gen_tok(full_prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        gen_out = gen_mod.generate(**gen_inp, max_new_tokens=150, do_sample=False, num_beams=1)

    sql = gen_tok.decode(gen_out[0], skip_special_tokens=True).split("```sql")[-1].split("```")[0].strip()
    if "SELECT" in sql: sql = "SELECT" + sql.split("SELECT", 1)[1]

    duration = time.time() - start_time

    # 시연용 출력 (SQL 컬러 강조)
    print(f"\n[Question {i+1}] {question}")
    print(f" ➤ Detected Table: {final_cands[0]['t_en']} ({final_cands[0]['t_ko']})")
    # Cyan Color for SQL
    print(f" ➤ Generated SQL: \033[1;96m{sql}\033[0m")
    print(f" ➤ Processing Time: {duration:.2f} sec")
    print("-" * 70)

    # 영상에서 읽을 시간 확보
    time.sleep(0.5)

print("\n✨ Demo Finished! (Perfect Version)")


🎬 [Demo] Text-to-SQL 파이프라인 (Final V5 - Dictator Mode)
   - Device: cuda
   - Model: defog/sqlcoder-7b-2
------------------------------------------------------------
📂 모델 로드 중...


The tokenizer you are loading from '/content/drive/MyDrive/P02_SemanticParsing/saved_models_final/schema_linker_e5/epoch_1' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ 준비 완료! (Prompt V5 Applied)

🚀 [Live Demo] Generating SQL...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...


[Question 8] 가로수길약국의 월요일과 화요일의 진료 시간을 알려줘
 ➤ Detected Table: TB_PHARMACY_OPERATE_INFO (서울시 약국 운영시간 정보)
 ➤ Generated SQL: SELECT p.dutytime1s, p.dutytime1c, p.dutytime7s, p.dutytime2c FROM tb_pharmacy_operate_info p WHERE p.dutyname ilike '%가로수길약국%'
 ➤ Processing Time: 2.68 sec
----------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



[Question 9] 주소에 강남대로가 들어가는 약국의 이름을 알려줘
 ➤ Detected Table: TB_PHARMACY_OPERATE_INFO (서울시 약국 운영시간 정보)
 ➤ Generated SQL: SELECT DUTYNAME FROM TB_PHARMACY_OPERATE_INFO WHERE DUTYADDR ilike '%강남대로%'
 ➤ Processing Time: 1.45 sec
----------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



[Question 10] 주소에 동광로가 들어가는 구심약국의 작업 일시를 알려줘
 ➤ Detected Table: TB_LIVE_PRIOR_PARKING_ZONE (서울시 거주자우선주차정보 표준 데이터)
 ➤ Generated SQL: SELECT TIME FROM TB_LIVE_PRIOR_PARKING_ZONE WHERE DUTYADDR LIKE '%동광로%'
 ➤ Processing Time: 1.42 sec
----------------------------------------------------------------------

✨ Demo Finished! (Perfect Version)
